In [342]:
import pandas as pd
import numpy as np
import re
#pd.set_option('display.max_rows',None)
# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

# Question 1

In [343]:
Energy = pd.read_excel('assets/Energy Indicators.xls', header = None, usecols='C:F', skiprows = 18, skipfooter = 38)
cols = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
Energy.columns = cols
Energy.set_index('Country', inplace=True)
ren  = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}


In [344]:
def conversion(item):
    if item == '...':
        return None
    else:
        new_val =  item*1000000
        return new_val

In [345]:
Energy['Energy Supply'] = Energy['Energy Supply'].apply(conversion)

In [346]:
def cleaning(item):

    result = re.sub('\d', '', item) #remove digits
    result = re.sub(r'\([^)]*\)', '', result) #Remove things in parenthesis
    result = result.rstrip()
    
    return result

Energy.index = Energy.index.map(cleaning)
Energy.rename(index = ren, inplace = True)


In [347]:
GDP = pd.read_csv('assets/world_bank.csv', skiprows=4, index_col= 0)

In [348]:
rena = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
dropcol =  list(range(2006,2016))
dropcol = map(str, dropcol)  
#GDP.drop(GDP.columns.difference(dropcol), 1, inplace= True)
GDP = GDP[dropcol]
GDP.index.rename('Country', inplace = True)


In [349]:
GDP.rename(index = rena, inplace = True)
print(GDP.shape)
GDP.head()


(264, 10)


,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,
Aruba,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Afghanistan,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Angola,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Albania,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


In [350]:
ScimEn = pd.read_csv('assets/scimagojr-3.csv')
ScimEn.head()


,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


In [351]:
ScimEn_short = ScimEn[ScimEn.index < 15]
new_df = ScimEn_short.merge(GDP, how = 'inner', left_on = 'Country', right_on = 'Country')
new_df.set_index('Country')
new_df = new_df.merge(Energy, how = 'inner', left_on='Country', right_on = 'Country')
new_df.set_index('Country', inplace = True)


In [352]:
sortby = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations','Citations per document', 'H index', 'Energy Supply','Energy Supply per Capita', '% Renewable','2006', '2007', '2008','2009', '2010', '2011', '2012', '2013','2014', '2015']
new_df = new_df[sortby]


In [353]:
entries = list()
entries.append(ScimEn.shape[0])
entries.append(GDP.shape[0])
entries.append(Energy.shape[0])
entries

[191, 264, 227]

# Part 3

In [354]:
years = list(range(2006,2016))
years = list(map(str,years))


In [355]:
new_df['avg'] = new_df[years].mean(axis = 1)
new_df.sort_values('avg', axis = 0,ascending = False, inplace= True)
gdp_serie = new_df['avg']
gdp_serie

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avg, dtype: float64

# Part 4

In [356]:
best6 = gdp_serie.index.values[5] ## return the country
df_6 = new_df.loc[best6]


In [357]:
gdp_trend = df_6.iloc[10:20]
min_gdp = gdp_trend[0]
max_gdp = gdp_trend[-1]
gdp_change = max_gdp - min_gdp
gdp_change

246702696075.3999

# Part 5

In [358]:
new_df.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,avg
Country,,,,,,,,,,,,,,,,,,,,,
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.57098,...,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,1.536434e+13
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.75491,...,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,6.348609e+12
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.23282,...,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,5.542208e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.90153,...,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,3.493025e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.02028,...,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,2.681725e+12


In [359]:
espc = new_df['Energy Supply per Capita']
ans = espc.mean()

# Part 6

In [360]:
new_df.sort_values('% Renewable',ascending=False, inplace=True)
most_ren = new_df.iloc[0]
country = most_ren.name
share = most_ren['% Renewable']
ans = (country, share)
ans


('Brazil', 69.64803)

# Part 7

In [361]:
def cit_ratio(item):
    nom =  item['Self-citations']
    den = item['Citations']
    ratio = nom/den
   
    return ratio

new_df['Citation Ratio'] = new_df.apply(cit_ratio, axis = 1)
new_df.sort_values('Citation Ratio', ascending=False,inplace = True)
top_ratio  = new_df.iloc[0]
country  = top_ratio.name
ratio  = new_df.at[country,'Citation Ratio']


# Part 8

In [362]:
def pop_est(item):
    e_supp = item['Energy Supply']
    e_p_cap = item['Energy Supply per Capita']
    pop_est = e_supp / e_p_cap
    return pop_est

new_df['Pop. Est.']  = new_df.apply(pop_est, axis = 1)

In [363]:
new_df.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2009,2010,2011,2012,2013,2014,2015,avg,Citation Ratio,Pop. Est.
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910,...,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,6.348609e+12,0.689313,1.367645e+09
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680,...,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,1.565459e+12,0.362517,1.435000e+08
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980,...,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,1.536434e+13,0.335031,3.176154e+08
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119,5.707721,...,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN,4.441558e+11,0.332782,7.707563e+07
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080,...,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,1.769297e+12,0.288973,1.276731e+09


In [364]:
third_pop = new_df.sort_values('Pop. Est.', ascending= False).iloc[3].name

# Part 9

In [365]:
def doc_per_person(item):
    cit_doc = item['Citable documents']
    pop = item['Pop. Est.']
    ans = cit_doc/ pop
    return ans


In [366]:
new_df['Citable documents per Capita'] = new_df.apply(doc_per_person,axis= 1)

In [367]:
new_df.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2010,2011,2012,2013,2014,2015,avg,Citation Ratio,Pop. Est.,Citable documents per Capita
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910,...,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,6.348609e+12,0.689313,1.367645e+09,0.000093
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680,...,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,1.565459e+12,0.362517,1.435000e+08,0.000128
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980,...,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,1.536434e+13,0.335031,3.176154e+08,0.000298
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119,5.707721,...,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN,4.441558e+11,0.332782,7.707563e+07,0.000114
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080,...,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,1.769297e+12,0.288973,1.276731e+09,0.000012


In [368]:

new_df['Citable documents per Capita']=np.float64(new_df['Citable documents per Capita'])
new_df['Energy Supply per Capita']=np.float64(new_df['Energy Supply per Capita'])
new_df['Energy Supply per Capita'].corr(new_df['Citable documents per Capita'], method='pearson')


0.7940010435442942

# part 10

In [369]:
median = new_df['% Renewable'].median()
def checker(item):
    if item >= median:
        return 1
    else:
        return 0
new_df['% Ren. check'] = new_df['% Renewable'].apply(checker)

In [370]:
new_df.sort_values('Rank', ascending=True, inplace=True)
ans = new_df['% Ren. check']

# part 11

In [371]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

In [372]:
def grouping(item):
    continent = ContinentDict[item]
    return continent

In [373]:
new_df['Country_name'] =  new_df.index.values
new_df['Continent'] = new_df['Country_name'].apply(grouping)
new_df.drop('Country_name', axis = 1, inplace = True)
new_df.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2012,2013,2014,2015,avg,Citation Ratio,Pop. Est.,Citable documents per Capita,% Ren. check,Continent
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,...,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,6.348609e+12,0.689313,1.367645e+09,0.000093,1,Asia
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,...,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,1.536434e+13,0.335031,3.176154e+08,0.000298,0,North America
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,...,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,5.542208e+12,0.275997,1.274094e+08,0.000238,0,Asia
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,...,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,2.487907e+12,0.183773,6.387097e+07,0.000319,0,Europe
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,...,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,1.565459e+12,0.362517,1.435000e+08,0.000128,1,Europe


In [374]:
small_df = new_df.groupby('Continent').agg({'Pop. Est.':(np.sum,np.mean,np.std)})
small_df['size'] = new_df.groupby('Continent').size()

In [375]:
final_df = small_df['Pop. Est.']
append = small_df['size']
final_df['size'] = append
final_df =  final_df[['size','sum','mean','std']]
final_df

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


# Part 12

In [376]:
new_df['% Renewable'] = pd.cut(new_df['% Renewable'],5)
new_df
    

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2012,2013,2014,2015,avg,Citation Ratio,Pop. Est.,Citable documents per Capita,% Ren. check,Continent
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,"(15.753, 29.227]",...,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,6.348609e+12,0.689313,1.367645e+09,0.000093,1,Asia
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,"(2.212, 15.753]",...,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,1.536434e+13,0.335031,3.176154e+08,0.000298,0,North America
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,"(2.212, 15.753]",...,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,5.542208e+12,0.275997,1.274094e+08,0.000238,0,Asia
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,"(2.212, 15.753]",...,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,2.487907e+12,0.183773,6.387097e+07,0.000319,0,Europe
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,"(15.753, 29.227]",...,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,1.565459e+12,0.362517,1.435000e+08,0.000128,1,Europe
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,"(56.174, 69.648]",...,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,1.660647e+12,0.190369,3.523986e+07,0.000500,1,North America
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,"(15.753, 29.227]",...,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,3.493025e+12,0.195111,8.036970e+07,0.000209,1,Europe
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,"(2.212, 15.753]",...,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,1.769297e+12,0.288973,1.276731e+09,0.000012,0,Asia
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,"(15.753, 29.227]",...,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,2.681725e+12,0.218943,6.383735e+07,0.000203,1,Europe


In [377]:
num = new_df.groupby([new_df['Continent'], new_df['% Renewable']]).size()

In [378]:
indexNames =  num[num == 0].index

In [379]:
num.drop(indexNames, inplace=True)

In [384]:
num.index.names

FrozenList(['Continent', '% Renewable'])

In [385]:
num

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

# Part 13

In [380]:
PopEst = new_df['Pop. Est.'].apply(str)
PopEst

Country
China                 1367645161.2903225
United States         317615384.61538464
Japan                 127409395.97315437
United Kingdom        63870967.741935484
Russian Federation           143500000.0
Canada                 35239864.86486486
Germany                80369696.96969697
India                 1276730769.2307692
France                 63837349.39759036
South Korea           49805429.864253394
Italy                 59908256.880733944
Spain                   46443396.2264151
Iran                   77075630.25210084
Australia             23316017.316017315
Brazil                205915254.23728815
Name: Pop. Est., dtype: object